In [1]:
path = "C:/Users/Yakina/.cache/kagglehub/datasets/mdwaquarazam/agricultural-crops-image-classification/versions/1/Agricultural-crops/"

In [2]:
import os
import random
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import matplotlib.image as mpimg

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, BatchNormalization, Flatten
from tensorflow.keras.preprocessing.image import ImageDataGenerator , load_img
from tensorflow.keras.models import Sequential
from tensorflow.keras.initializers import HeNormal 
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
from tensorflow.keras.losses import SparseCategoricalCrossentropy

initializer = HeNormal()

In [ ]:
dirs = os.listdir(path)

for dir in dirs:
    files = list(os.listdir(path + dir))
    print(dir, len(files))

In [ ]:
train_ds = tf.keras.utils.image_dataset_from_directory(
    path,
    seed = 111,
    validation_split = 0.2,
    subset = 'training',
    batch_size = 32,
)

In [ ]:
val_ds = tf.keras.utils.image_dataset_from_directory(
    path,
    seed = 111,
    validation_split = 0.2,
    subset = 'validation',
    batch_size = 32,
)

In [ ]:
crops = train_ds.class_names

In [ ]:
len(crops)

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)

In [ ]:
# augmentation

data_augmentation = tf.keras.Sequential([
  layers.RandomFlip("horizontal_and_vertical"),
  layers.RandomRotation(0.2),
  layers.RandomZoom(0.2)
])

In [ ]:
model = Sequential([
    data_augmentation,
    layers.Rescaling(1./255),
    Conv2D(16, 3,padding = "same", activation='relu'),
    MaxPooling2D(),
    Conv2D(32, 3,padding = "same", activation='relu'),
    MaxPooling2D(),
    Conv2D(64, 3,padding = "same", activation='relu'),
    MaxPooling2D(),
    Dropout(0.2),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(len(crops))
])

In [ ]:
model.compile(optimizer = 'adam',
    loss = SparseCategoricalCrossentropy(from_logits=True),
    metrics = ['accuracy']
)

In [ ]:
model.summary()

In [ ]:
history = model.fit(train_ds, epochs=20, validation_data=val_ds)